# Introduction au traitement de texte

Vous avez beaucoup travaillé sur le traitement d'image dans les TPs précédents, on va maintenant s'intéresser au traitement de texte. Dans ce TP on va se concentrer sur de la classification de texte. Vous verrez dans les prochains TPs comment générer du texte, comment faire de la traduction automatique, etc.

Pour ce TP, on va utiliser le dataset `AG_NEWS`, qui contient des articles de journaux classés en 4 catégories : World, Sports, Business, Sci/Tech.

On va tout d'abord télécharger le dataset et voir comment il ressemble.



## Téléchargement du dataset

Avant de télécharger le dataset, on va d'abord installer les librairies `torchtext` et `portalocker`.

Si vous êtes sur colaboratory, vous pouvez ajouter une cellule de code et taper `!pip install torchtext portalocker` pour installer les librairies.

In [18]:
import torch
import gdown

url = "https://drive.google.com/uc?export=download&id=1fu5XViKfHsA5qpy_jLI7ddBb0Zl_Uout"
output = "ag_news.zip"

gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1fu5XViKfHsA5qpy_jLI7ddBb0Zl_Uout
To: /content/ag_news.zip
100%|██████████| 11.9M/11.9M [00:00<00:00, 30.5MB/s]


'ag_news.zip'

In [19]:
!unzip ag_news.zip

Archive:  ag_news.zip
  inflating: test.csv                
  inflating: train.csv               


In [11]:
# import into dataframes
import pandas as pd

train_df = pd.read_csv("train.csv", skiprows=1, names=["label", "title", "description"])
test_df = pd.read_csv("test.csv", skiprows=1, names=["label", "title", "description"])

## Exercice 1: Affichage du dataset
Essayez d'affichier quelques articles du dataset, et les labels associés. `train_dataset` est un itérable, donc vous pouvez itérer dessus avec une boucle `for`

,label,description
0,3,"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Reuters - Private investment firm Carlyle Grou...
2,3,Reuters - Soaring crude prices plus worries\ab...
3,3,Reuters - Authorities have halted oil export\f...
4,3,"AFP - Tearaway world oil prices, toppling reco..."
...,...,...
119995,1,KARACHI (Reuters) - Pakistani President Perve...
119996,2,Red Sox general manager Theo Epstein acknowled...
119997,2,The Miami Dolphins will put their courtship of...
119998,2,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


## L'idée de la méthode pour classifier du texte

Tout d'abord, on va split le texte en une liste de mots. Ensuite, on va représenter chaque mot par un vecteur. Puis on va faire la moyenne de tous les vecteurs de la phrase pour obtenir un vecteur de taille fixe qui représente le texte qu'on pourrait le faire passer dans un réseau de neurones pour classifier le texte.

Pourquoi faire la moyenne de tous les vecteurs?

- Parce que si on a une phrase avec 3 mots, on va avoir 3 vecteurs pour représenter cette phrase. Si on a une autre phrase avec 5 mots, on va en avoir 5 pour représenter la phrase.
	Donc si on veut faire passer ces phrases dans un réseau de neurones, on va avoir des vecteurs de taille différente en entrée, et ça ne va pas marcher car les couches linéaires veulent des vecteurs de taille fixe en entrée. (Si vous vous souvenez bien, il faut préciser `input_dim` pour `nn.Linear`). Il existe des méthodes pour faire passer des vecteurs de taille variable dans un réseau de neurones, mais vous les verrez dans les prochains TPs.


Mais comment représenter un mot par un vecteur? Et plus précisément par quel vecteur? Comment on peut déterminer les valeurs de ces vecteurs?
- Ça va être difficile de déterminer les valeurs de ces vecteurs par nous-même, mais on peut les apprendre! On va initialiser les vecteurs avec des valeurs aléatoires, et on va les modifier pendant l'entraînement de notre réseau de neurones qui classifie le texte avec la backprop.

Donc chaque mot va être représenté par un vecteur de poids de taille `D`, qui est initialisé aléatoirement. Et on va avoir `V` mots dans notre vocabulaire. On va donc avoir une matrice de taille `VxD` qui contient les vecteurs de tous les mots de notre vocabulaire. Ces vecteurs s'appellent d'ailleurs des `embeddings`.

On pourrait créer manuellement cette matrice, mais il existe une couche dans PyTorch qui va le faire pour nous : `nn.Embedding`. Cette couche prend en entrée un entier, et retourne le vecteur associé à cet entier. Cette couche va aussi apprendre les vecteurs durant l'entraînement, c'est-à-dire apprendre les poids (les valeurs) de chaque vecteur.

Ici, ces entiers correspondent à l'index du mot dans le vocabulaire. On va donc devoir associer un index à chaque mot de notre vocabulaire.
Tout d'abord, on va utiliser un tokenizer, qui va prendre en entrée une phrase, et va retourner une liste de mots. L'avantage d'un tokenizer c'est qu'il va aussi enlever les majuscules, les caractères spéciaux, etc. On va utiliser le tokenizer de `tokenizers` (une librairie de hugging face), qui est un tokenizer très simple.
On va ensuite créer un vocabulaire, qui va associer un index à chaque mot. Ce vocabulaire servira pour transformer un texte en une liste d'indices, et qui ensuite sera envoyée dans la couche `nn.Embedding` pour retourner les vecteurs associés à ces indices.

![Text Classify](https://raw.githubusercontent.com/Automatants/projets-de-permanence/master/image-hosting/TP7_Intro_NLP/textclassify.png)

Donc pour résumer:
- On va créer un tokenizer et un vocabulaire, pour associer un index à chaque mot.
- On va créer une couche `nn.Embedding` qui prend en entrée des indices, et retourne les vecteurs associés à ces indices.
- On va créer un réseau de neurones, qui prend en entrée la moyenne des vecteurs de tous les mots de la phrase, et qui prédit la classe du texte.
- On va entraîner le réseau de neurones, et les vecteurs de la couche `nn.Embedding` vont être appris pendant l'entraînement.

## Création du tokenizer et vocabulaire
On va utiliser `get_tokenizer` pour créer un tokenizer. Vous pouvez avoir plus d'informations sur cette fonction ici : https://pytorch.org/text/stable/data_utils.html

In [1]:
from tokenizers import Tokenizer, models

tokenizer = Tokenizer.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

On peut maintenant utiliser le vocabulaire pour transformer un texte en une liste d'index.

In [3]:
encoded = tokenizer.encode("Hello woRld")
print(encoded.tokens)
print(encoded.ids)

['[CLS]', 'hello', 'world', '[SEP]']
[101, 7592, 2088, 102]


Je sais que tout ça n'est pas encore très clair, mais vous allez pouvoir vous entraîner sur le challenge après ce TP.

## Préparation du dataset d'entraînement
Tout d'abord, on va convertir notre `train_dataset` en un dataset de liste d'index de mots.

In [15]:
X_train = []
Y_train = []

# on enlève le 'title'
#train_df = train_df.drop(columns=["title"])

# on veux que les labels soient en int
train_df["label"] = train_df["label"].astype(int)

# remarque, les labels sont entre 1 et 4, mais on veut qu'ils soient entre 0 et 3
Y_train = train_df["label"] - 1

for index, row in train_df.iterrows():
	tokens = tokenizer.encode(row["description"]).ids
	X_train.append(tokens)

print(X_train[0])
print(Y_train[0])

print(X_train[1])
print(Y_train[1])

[101, 26665, 1011, 2460, 1011, 19041, 1010, 2813, 2395, 1005, 1055, 1040, 11101, 2989, 1032, 2316, 1997, 11087, 1011, 22330, 8713, 2015, 1010, 2024, 3773, 2665, 2153, 1012, 102]
2
[101, 26665, 1011, 2797, 5211, 3813, 18431, 2571, 2177, 1010, 1032, 2029, 2038, 1037, 5891, 2005, 2437, 2092, 1011, 22313, 1998, 5681, 1032, 6801, 3248, 1999, 1996, 3639, 3068, 1010, 2038, 5168, 2872, 1032, 2049, 29475, 2006, 2178, 2112, 1997, 1996, 3006, 1012, 102]
2


Maintenant, on aimerait convertir X_train en un `torch.tensor`, mais on a un problème : les phrases n'ont pas toutes la même longueur, donc on ne peut pas créer un `torch.tensor` avec toutes les phrases. On va donc devoir "padder" les phrases, c'est-à-dire ajouter des "\<pad>" à la fin des phrases pour qu'elles aient toutes la même longueur. On va donc calculer la longueur maximale des phrases et mettre du padding à la fin des phrases qui sont plus courtes que la longueur maximale.

### Exercice:
Mettre des `vocab["<pad>"]` à la fin de chaque élément de `X_train` pour que tous les éléments de `X_train` aient la même longueur.

In [16]:
tokenizer.add_tokens(["<pad>"])

1

In [ ]:
...

print(X_train[0])
print(X_train[1])

[431, 425, 1, 1605, 14838, 113, 66, 2, 848, 13, 27, 14, 27, 15, 50725, 3, 431, 374, 16, 9, 67507, 6, 52258, 3, 42, 4009, 783, 325, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[15874, 1072, 854, 1310, 4250, 13, 27, 14, 27, 15, 929, 797, 320, 15874, 98, 3, 27657, 28, 5, 4459, 11, 564, 52790, 8, 80617, 2125, 7, 2, 525, 241, 3, 28, 3890, 82814, 6574, 10, 206, 359, 6, 2, 126, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Cool, maintenant on peut créer un `torch.tensor` avec toutes les phrases.

In [17]:
X_train_torch = torch.tensor(X_train)
Y_train_torch = torch.tensor(Y_train)

print(X_train_torch.shape)
print(Y_train_torch.shape)

NameError: name 'torch' is not defined

## Création du réseau de neurones
Notre réseau de neurones va prendre en entrée un `torch.tensor` de taille `(batch_size, max_sentence_length)` qui correspond à un batch de phrases tokenisées et transformées en liste d'index de mots, et va retourner un `torch.tensor` de taille `(batch_size, 4)`, où chaque ligne correspond à la probabilité d'appartenance à chaque classe.
Le réseau contient une couche d'embedding, qui va transformer les index des mots en vecteurs de taille `embedding_dim` (ou `D` comme on a dit plus haut), et deux couches 2 linéaires.
On va devoir choisir la taille de nos vecteurs d'embedding `embedding_dim`.

In [ ]:
import torch.nn as nn

class Net(nn.Module):
	def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
		super().__init__()
		self.embedding = nn.Embedding(vocab_size, embedding_dim)
		self.fc1 = nn.Linear(embedding_dim, hidden_dim)
		self.fc2 = nn.Linear(hidden_dim, output_dim)

		self.relu = nn.ReLU()

	def forward(self, x):
		# x.shape = (batch_size, max_sentence_length)
		embedded = self.embedding(x)
		# embedded.shape = (batch_size, max_sentence_length, embedding_dim)
		embedded = embedded.mean(dim=1)
		# embedded.shape = (batch_size, embedding_dim)
		out = self.fc1(embedded)
		out = self.relu(out)
		out = self.fc2(out)
		return out

Comme vous pouvez voir dans le `forward`, on utilise la couche d'embedding, pour transformer les index des mots en vecteurs de taille `embedding_dim`, puis on calcule la moyenne sur la dimension des phrases pour avoir un vecteur de taille fixe `embedding_dim` pour chaque phrase afin de le passer dans les couches linéaires.

Mais on ne veut pas prendre en compte les vecteurs qu'on a ajouté pour le padding, donc on va devoir modifier le `forward` pour ne pas les prendre en compte.
Comment faire ça?
On va créer un `mask` qui va être un `torch.tensor` de taille `(batch_size, max_sentence_length)` qui vaut `True` si le mot est un mot du vocabulaire et `False` si le mot est un padding.

On va ensuite multiplier `embedded` par `mask.float()` pour mettre à 0 les vecteurs des mots qui sont des paddings.

Mais, il y a un problème:
- `mask` est de taille `(batch_size, max_sentence_length)`
- `embedded` est de taille `(batch_size, max_sentence_length, embedding_dim)`

donc on ne peut pas les multiplier. On va donc devoir modifier `mask` pour qu'il soit de taille `(batch_size, max_sentence_length, embedding_dim)`.

Pour ça, on va utiliser la fonction `unsqueeze` (documentation: https://pytorch.org/docs/stable/generated/torch.unsqueeze.html) qui permet d'ajouter une dimension à un `torch.tensor`. Donc notre `mask` de taille `(batch_size, max_sentence_length)` va devenir de taille `(batch_size, max_sentence_length, 1)`.

Comme `torch` est intelligent, lorsque qu'on fait `embedded * mask.float()`, il va automatiquement répéter `mask` sur la dimension `embedding_dim` pour qu'il soit de taille `(batch_size, max_sentence_length, embedding_dim)`. Cette opération s'appelle le `broadcasting`.

Une fois qu'on a fait ça, on peut calculer la moyenne sur la dimension des phrases, et on obtient un `torch.tensor` de taille `(batch_size, embedding_dim)` qu'on peut passer dans les couches linéaires.

**Attention**: si on utilise `torch.mean`, il va diviser par la longueur maximale des phrases (donc il prend en compte les paddings, alors qu'on ne veut pas les prendre en compte; on veut diviser par la vraie longueur de chaque phrase), donc on va devoir utiliser `torch.sum` et diviser par le nombre de mots qui ne sont pas des paddings.

Pour compter le nombre de mots qui ne sont pas des paddings, on peut utiliser le `mask` qu'on a créé. En effet, `mask.float()` vaut `1` si le mot est un mot du vocabulaire et `0` si le mot est un padding.

**Remarque**: Lorsque vous faites `torch.sum()`, ou `torch.mean()` sans préciser `dim`, il va calculer la somme ou la moyenne sur tous les éléments du `torch.tensor`, et donc vous obtiendrez un `torch.tensor` de taille `(1,)`, autrement dit un scalaire. Si vous voulez calculer la somme ou la moyenne sur une dimension, il faut préciser `dim` (par exemple `torch.mean(x, dim=1)` ou `x.mean(dim=0)`).
Plus d'information sur la documentation: https://pytorch.org/docs/stable/generated/torch.mean.html et https://pytorch.org/docs/stable/generated/torch.sum.html

In [ ]:
import torch.nn as nn

class Net(nn.Module):
	def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
		super().__init__()
		self.embedding = nn.Embedding(vocab_size, embedding_dim)
		self.fc1 = nn.Linear(embedding_dim, hidden_dim)
		self.fc2 = nn.Linear(hidden_dim, output_dim)

		self.relu = nn.ReLU()

	def forward(self, x):
		# x.shape = (batch_size, max_sentence_length)

		embedded = self.embedding(x)
		# embedded.shape = (batch_size, max_sentence_length, embedding_dim)

		mask = (x != vocab["<pad>"])
		# mask.shape = (batch_size, max_sentence_length)

		...
		# mask.shape = (batch_size, max_sentence_length, 1)

		embedded = ... # multiplier embedded par mask.float() pour mettre à 0 les vecteurs des mots qui sont des paddings
		# embedded.shape = (batch_size, max_sentence_length, embedding_dim)

		# calculer la moyenne
		...

		out = self.fc1(embedded)
		out = self.relu(out)
		out = self.fc2(out)
		return out

## Entraînement
La boucle d'entraînement est la même que pour les TPs précédents, donc à vous de jouer! Utilisez la loss `nn.CrossEntropyLoss` et l'optimiseur `torch.optim.Adam`. N'oubliez pas de batcher les données et d'entraîner sur le gpu.

In [ ]:
...

## Évaluation sur l'accuracy

In [ ]:
...

## Amélioration possible
Vous pouvez remarquer que le tokenizer n'enlève pas les ponctuations, donc vous pouvez essayer de les enlever pour voir si ça améliore les performances.

## Challenge: Analyse de sentiments
Lien vers le challenge: https://sharing.cs-campus.fr/compete/100